![Imandrabot](../images/kostya_ros_medium_1.png)

*At AI, we've been working on an IML (Imandra Modelling Language) interface to ROS, allowing one to develop ROS nodes and use Imandra to verify their properties. In this notebook, we will go through creation and verification of a Robotic Operating System (ROS) node in Imandra. We will make a robot control node that controls the motion of a simple 2-wheeler bot:*

![Imandrabot](../images/Imandrabot.png)

We'll create a controller that uses the laser scanner to avoid obstacles and drive around the scene. The Imandra ML code can be compiled in OCaml and plugged into the ROS system - the behaviour of the bot can be observed in the Gazebo simulator.

Then we'll illustrate how to use Imandra to formally verify various statements about the model and how to find bugs and corner cases by exploring the Imandra-generated counterexamples for false conjectures.

# 1. ROS message OCaml types

For our Imandra-ROS project we’ve processed all the standard ROS messages with our code generation tool creating a collection of strongly-typed IML/OCaml bindings for them. But, in order to keep this notebook self-contained we'll define the necessary messaging modules here.  

First, we'll need to declare the message type that will control our robot. This is typically done with a `Twist` message from the `geometry_msgs` standard ROS package. We want to mimic ROS messaging nomenclauture as close as possible, so we'll create an OCaml/Imadra `module` with the same name as the package and will place the necessary type/message declaraions inside:

In [1]:
module Geometry_msgs = struct
  type vector3 = 
    { vector3_x : int
    ; vector3_y : int
    ; vector3_z : int
    }
  type twist = 
    { twist_linear  : vector3
    ; twist_angular : vector3
    }    
end

module Geometry_msgs :
  sig
    type vector3 = { vector3_x : int; vector3_y : int; vector3_z : int; }
    type twist = { twist_linear : vector3; twist_angular : vector3; }
  end


You might have noticed that we've replaced floating point values for vector coordinates with integers. In this context, it is more straight-forward for Imandra to reason about integers, so we assume that there is a common factor of 100,000 multiplying all the incoming floating point values and divides all the outgoing integers. (That effectively makes our unit of measurement of length to be 10 micrometres).

Let's move on and declare the incoming messages: 
 - `LaserScan` sensor input message from the `sensor_msgs` ROS package
 - and the `Clock` message from the `Rosgraph_msg` ROS package  
 
We define the wrapping modules for both messages and declare their datatypes:

In [3]:
module Sensor_msgs = struct
  type laserScan = 
    { laserScan_range_min : int 
    ; laserScan_range_max : int 
    ; laserScan_ranges : int list 
    }
end
module Rosgraph_msgs = struct
  type time = 
    { seconds     : int
    ; nanoseconds : int
    }
  type clock = { clock : time }
end

module Sensor_msgs :
  sig
    type laserScan = {
      laserScan_range_min : int;
      laserScan_range_max : int;
      laserScan_ranges : int list;
    }
  end
module Rosgraph_msgs :
  sig
    type time = { seconds : int; nanoseconds : int; }
    type clock = { clock : time; }
  end


Robotics OS middleware will communicate with our node via messages of these three types. The code that we'll write for of our node will represent the formal mathematical model of the controller algorithm - we can use Imandra to reason and verify various statements about the code. Since IML is valid OCaml, we'll leverage its compiler to create an executable from the verified IML code.

# 2. Creating a simple ROS Node model

We want to create some simple but non-trivial robot controller that makes our bot drive around avoiding the obstacles. The bot is going to drive forward until one of the laser scanner ranges becomes too low, meaning that we've gotten too close to some obstacle - in that case, we want the bot to stop and turn until the scanner tells us that the road ahead is clear. To make the model a bit more complicated, we'd like to implement the ability to choose the turning direction depending on the laser scanner ranges.

One might try to make a "naive" controller that doesn't have any memory about its previous states and observations - such a bot reacts to the currently observed scanner values and decides its actions based solely on that information. Such an approach will quickly lead to the bot being "stuck" in infinite oscillatory loops. E.g. here is a bot that decides which side to turn depending on the first value in the `ranges` array:

![Imandrabot](../images/Stuck.gif)

To avoid this kind of oscillations we need the model to have some memory of its previous states. The idea is to introduce two modes of model's operation: driving forward and turning in one place. The bot is in the "driving" mode by default, but it can transition to the turning mode if it gets dangerously close to surrounding objects.

The turning direction is calculated using the direction of the minimum of the distances that the scanner returns. While the robot is turning in one place, it stores the minimal range that the scanner has detected at that location. If at some point the scanner detects a range that is lower than the stored one - the turning direction gets recalculated, and the minimal range gets updated.

## 2.1 State datatype

Working with Imandra we’ve adopted a standard way to construct formal models of message-driven systems. At the top of the model we have a single OCaml datatype that holds all the data needed to describe the system at a given moment, including incoming and outgoing messages. We call this record type `state`. Together with this `state` type we define a `one_step` transition `state -> state` function, which performs a single logically isolated step of the simulation and returns the new `state` after the transition.

As an example, consider an IML/OCaml type declaration for a simple ROS node that is able to accept `rosgraph_msgs/Clock` and `sensor_msgs/LaserScan` standard ROS messages. We also want the state to store three values: 
 - the current mode of the bot -- whether we are driving forward or turning in a preferred direction
 - the latest minimal value of the ranges that the laser sensor returns
 - the preferred side for the robot to turn -- either clockwise (`CW`) or counter-clockwise (`CCW`) 
 
Finally, we want the node to be able to send `geometry_msgs/Twist` ROS message depending on the stored `min_range` data:

In [5]:
type incoming_msg = 
  | Clock  of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan
  
type outgoing_msg =
  | Twist of Geometry_msgs.twist

type direction = CW | CCW 

type mode = Driving | Turning

type state =
  { mode : mode
  ; min_range : int option
  ; direction : direction option
  ; incoming  : incoming_msg option
  ; outgoing  : outgoing_msg option 
  }

type incoming_msg =
    Clock of Rosgraph_msgs.clock
  | Sensor of Sensor_msgs.laserScan
type outgoing_msg = Twist of Geometry_msgs.twist
type direction = CW | CCW
type mode = Driving | Turning
type state = {
  mode : mode;
  min_range : int option;
  direction : direction option;
  incoming : incoming_msg option;
  outgoing : outgoing_msg option;
}


## 2.2 State transition `one_step` function

To implement our node, we'll need a function that scans through a list of values and returns the minimum value and its index. We'll make a generic function `foldi` that does an indexed version of the `List.fold_right`:

In [6]:
let rec foldi ~base ?(i=0) f l =
  match l with
  | [] -> base
  | x :: tail -> f i x ( foldi f ~base ~i:(i+1) tail )

val foldi : base:'a -> ?i:Z.t -> (Z.t -> 'b -> 'a -> 'a) -> 'b list -> 'a =
  <fun>


termination proof Termination proof call `foldi base (Some ((if Is_a(Some, *opt*) then Option.get *opt* else 0) + 1))
f_2 (List.tl l)` from `foldi base *opt* f_2 l` original foldi base *opt* f_2 l sub foldi base (Some ((if Is_a(Some, *opt*) then Option.get *opt* else 0) + 1))
f_2 (List.tl l) original ordinal Ordinal.Int (Ordinal.count l) sub ordinal Ordinal.Int (Ordinal.count (List.tl l)) path [not (l = [])] proof detailed proof summary full ground_instances 3 definitions 0 inductions 0 search_time 0.010s details Expand smt_stats num checks 7 arith assert lower 6 arith pivots 5 rlimit count 1908 mk clause 3 datatype occurs check 43 mk bool var 54 arith assert upper 6 datatype splits 3 decisions 7 arith add rows 15 propagations 2 conflicts 7 arith fixed eqs 5 datatype accessor ax 5 arith conflicts 1 datatype constructor ax 8 num allocs 562868257 final checks 6 added eqs 34 del clause 1 arith eq adapter 4 memory 11.290000 max memory 19.550000 Expand start[0.010s]
 not (l = []) && Ordinal.count l >= 0 && Ordinal.count (List.tl l) >= 0
 ==> List.tl l = []
 || Ordinal.Int (Ordinal.count (List.tl l)) Ordinal.<<
 Ordinal.Int (Ordinal.count l) simplify into (not
 ((not (l = []) && Ordinal.count l >= 0) && Ordinal.count (List.tl l) >= 0)
 || List.tl l = [])
|| Ordinal.Int (Ordinal.count (List.tl l)) Ordinal.<<
 Ordinal.Int (Ordinal.count l) expansions [] rewrite_steps forward_chaining unroll expr (Ordinal.<<_131 (Ordinal.Int_122 (|count_`ty_1 list`_2255|
 (|get.… expansions unroll expr (|count_`ty_1 list`_2255| (|get.::.1_2233| l_2244)) expansions unroll expr (|count_`ty_1 list`_2255| l_2244) expansions unsat (let ((a!1 (ite (>= (|count_`ty_1 list`_2255| (|get.::.1_2233| l_2244)) 0)
 (|count_`…

When accepting this function, Imandra constructs its "termination proof" - that means that Imandra managed to prove that recursive calls in this function will not end up in an infinite loop. Imandra proves such things using inductive reasoning and is able to prove further statements about other properties of such functions.

In [7]:
let get_min_range max lst =
  List.fold_right ~base:max
    (fun x a -> if x < a then x else a) lst

val get_min_range : int -> int list -> int = <fun>


On an incoming `Clock` tick we are simply sending out a `Twist` message which tells the robot to either move forward or turn, depending on the mode that it is currently in. We encode it by introducing the `make_twist_message` helper function and the `process_clock_message : state -> state` function.

In [8]:
let make_twist_message v omega=
  let open Geometry_msgs in
  let mkvector x y z =  { vector3_x = x; vector3_y = y; vector3_z = z   } in 
  Twist { twist_linear  = mkvector v 0 0 ; twist_angular = mkvector 0 0 omega }

let process_clock_message state =
  match state.mode with 
  | Driving -> { state with outgoing = Some (make_twist_message 10000 0) } 
  | Turning -> begin
  match state.direction with 
    | None
    | Some ( CW ) -> { state with outgoing = Some (make_twist_message 0   10000) } 
    | Some (CCW ) -> { state with outgoing = Some (make_twist_message 0 (-10000))} 
  end

val make_twist_message : int -> int -> outgoing_msg = <fun>
val process_clock_message : state -> state = <fun>


On incoming `Scan` message, we want to find the minimum of the received ranges and the index of that minimum in the list. Depending on the index, we decide in which direction to turn. To implement this, we create another helper function and the `process_sensor_message` one:

In [9]:
let get_min_and_direction msg =
  let max = msg.Sensor_msgs.laserScan_range_max in
  let lst = msg.Sensor_msgs.laserScan_ranges in
  let min_range = get_min_range max lst in
  let mini = foldi ~base:(max, 0)
    (fun i a b -> if a < fst b then (a,i) else b) in
  let _ , idx = mini lst in
  if idx < List.length lst / 2 then min_range, CW else min_range, CCW

let process_sensor_message state min_range min_direction =
  let dirving_state = 
    { state with mode = Driving; min_range = None; direction = None } in
  let turning_state = 
    { state with 
      mode      = Turning
    ; direction = Some min_direction
    ; min_range = Some min_range 
    } in  
  match state.mode , state.min_range with 
  | Driving , _    -> if min_range < 20000 then turning_state else dirving_state
  | Turning , None -> if min_range > 25000 then dirving_state else turning_state
  | Turning , Some old_range -> 
    if min_range > 25000 then dirving_state
    else if min_range > old_range then state else turning_state

val get_min_and_direction : Sensor_msgs.laserScan -> int * direction = <fun>
val process_sensor_message : state -> int -> direction -> state = <fun>


With the help of these functions, we can create our `one_step` transition function, which just dispatches the messages to the appropriate helper function above.

In [11]:
let one_step state =
  match state.incoming with None -> state | Some in_msg ->
  let state = { state with incoming = None; outgoing = None } in
  match in_msg with 
  | Sensor laserScan -> 
    let min_range, min_direction = get_min_and_direction laserScan in
    process_sensor_message state min_range min_direction
  | Clock  _ -> process_clock_message state

val one_step : state -> state = <fun>


## 2.3 Running the model as a ROS node

Now that we have an early model, let's compile it with our ROS node wrapper into an executable. Here is the model, controlling our "imandrabot" in the Gazebo simulation environment:

![Imandrabot](../images/Imandra_Demo.gif)

# 3. Verifying the ROS node model

Formal verification is the process of reasoning mathematically about the correctness of computer programs. We'll use Imandra to formally verify some properties of the ROS node model we've created.

## 3.1 Verifying outgoing `Twist` message at `Clock` ticks 

Our model is designed in such a way that it updates its state parameters upon `LaserScan` messages and sends out `Twist` control messages in response to `Clock` messages. Let's verify a simple theorem that on every incoming `Clock` message, there is an outgoing `Twist` message. 

We can formally write this statement down as:

$$ \forall s. IsClock(IncomingMessage(s)) \,\Rightarrow\, IsTwist(OutgoingMessage(OneStep(s))) $$

eaning that for every state $s$, if the state contains an incoming message and this message is a `Clock` message, then the state's `outgoing` message is a `Twist` after we've called `one_step` on it.  

We can almost literally encode this formal expression as an Imandra `theorem`:

In [13]:
let is_clock msg = match msg with  Some ( Clock _ ) -> true | _ -> false ;;
let is_twist msg = match msg with  Some ( Twist _ ) -> true | _ -> false ;;

theorem clock_creates_outbound state =
  is_clock state.incoming ==> is_twist (one_step state).outgoing

val is_clock : incoming_msg option -> bool = <fun>
val is_twist : outgoing_msg option -> bool = <fun>
val clock_creates_outbound : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.008s details Expand smt_stats num checks 2 arith assert lower 2 rlimit count 1799 mk clause 39 mk bool var 209 arith assert upper 2 datatype splits 26 decisions 17 propagations 35 conflicts 10 datatype accessor ax 47 datatype constructor ax 33 num allocs 642576992 added eqs 284 del clause 3 arith eq adapter 1 memory 12.700000 max memory 19.550000 Expand start[0.008s]
 (if Is_a(Clock, Option.get :var_0:.incoming)
 && Is_a(Some, :var_0:.incoming)
 then true else false)
 ==> (if Is_a(Twist,
 Option.get
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 && Is_a(Some,
 (if :var_0:.incoming = None then :var_0: else …).outgoing)
 then true else false) simplify into not
(Is_a(Clock, Option.get :var_0:.incoming) && Is_a(Some, :var_0:.incoming))
|| Is_a(Twist,
 Option.get (if :var_0:.incoming = None then :var_0: else …).outgoing)
 && Is_a(Some, (if :var_0:.incoming = None then :var_0: else …).outgoing) expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2438 (get.Some.0_2440 (inco… digraph "proof" {
p_208 [label="Start ((if Is_a(Clock, Option.get :var_0:.incoming)\l && Is_a(Some, :var_0:.incoming)\l then true else false)\l ==\> (if Is_a(Twist,\l Option.get\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l && Is_a(Some,\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l then true else false)\l :time 0.008s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_208 -> p_207 [label=""];
p_207 [label="Simplify (not\l (Is_a(Clock, Option.get :var_0:.incoming)\l && Is_a(Some, :var_0:.incoming))\l \|\| Is_a(Twist,\l Option.get\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l && Is_a(Some,\l (if :var_0:.incoming = None then :var_0: else …).outgoing)\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_207 -> p_206 [label=""];
p_206 -> p_z3_1031 [label="",fontname="courier",fontsize=14];
p_z3_1031 [label="(let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2438 (get.Some.0_2440 (inco…",shape=box,fontname="courier",fontsize=14];p_206 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l(not (Is_a(Clock, Option.get state.incoming) && Is_a(Some, state.incoming))\l \|\| Is_a(Twist,\l Option.get (if state.incoming = None then state else …).outgoing)\l && Is_a(Some, (if state.incoming = None then state else …).outgoing))",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_292 [label="foldi ((Destruct(Sensor, 0, …)).Sensor_msgs.laserScan_range_max, 0) None\lanon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_292 [label="calls"];
call_295 [label="List.fold_right anon_fun.get_min_range.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_295 [label="calls"];
call_288 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_288 [label="calls"];
}
 Load graph Loading..

One can see that Imandra says that it "Proved" the theorem, meaning that Imandra has formally checked that this property holds for all possible input states. 

## 3.2 Verifying that we never drive backwards 

As another example, let us check that, no matter what, the node will never send out a message with negative linear velocity.

In [15]:
let no_moving_back msg = 
    let open Geometry_msgs in
    match msg with None -> true 
    | Some (Twist data) -> data.twist_linear.vector3_x >= 0
  
verify ( fun state -> no_moving_back (one_step state).outgoing  )

val no_moving_back : outgoing_msg option -> bool = <fun>
- : state -> bool = <fun>
module CX : sig val state : state end


We have failed to prove the statement and Imandra have created a counterexample `CX` module for us. This module  contains concrete values for the parameters of the verified statement, that violate the statement's condition. Let's examine the value of `CX.state`:

In [16]:
CX.state

- : state =
{mode = Turning; min_range = Some 2282; direction = None; incoming = None;
 outgoing =
  Some
   (Twist
     {Geometry_msgs.twist_linear =
       {Geometry_msgs.vector3_x = -1; vector3_y = 9; vector3_z = 10};
      twist_angular =
       {Geometry_msgs.vector3_x = 11; vector3_y = 12; vector3_z = 13}})}


The counterexample `state` produced by imandra has the `incoming` message set to `None` and the `outgoing` message set to a `Twist` message with negative `linear.x`. Our `one_step` function keeps the state unchanged if the incoming message is empty. 

We can either consider this behavior as a bug and change our `one_step` implementation; or we can consider this a normal behavior and amend our theorem, adding the non-empty incoming message as an extra premise of the theorem: 

In [19]:
theorem never_goes_back state = 
  state.incoming <> None
  ==>
  no_moving_back (one_step state).outgoing  

val never_goes_back : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.010s details Expand smt_stats num checks 2 arith assert lower 3 rlimit count 2372 mk clause 48 mk bool var 282 arith assert upper 14 datatype splits 28 decisions 56 propagations 162 conflicts 21 datatype accessor ax 57 minimized lits 7 arith assert diseq 2 datatype constructor ax 74 num allocs 814411800 added eqs 534 del clause 3 arith eq adapter 5 memory 26.710000 max memory 26.710000 Expand start[0.010s]
 not (:var_0:.incoming = None)
 ==> (if (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if ….mode = Driving then … else …
 else if ….mode = Driving then … else …).outgoing
 = None
 then true
 else
 (Destruct(Twist, 0, …)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x
 >= 0) simplify into (:var_0:.incoming = None
 || (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).outgoing
 = None)
|| (Destruct(Twist, 0,
 Option.get
 (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x
 >= 0 expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2821 (get.Some.0_2823 (inco… digraph "proof" {
p_232 [label="Start (not (:var_0:.incoming = None)\l ==\> (if (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if ….mode = Driving then … else …\l else if ….mode = Driving then … else …).outgoing\l = None\l then true\l else\l (Destruct(Twist, 0, …)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \>= 0)\l :time 0.010s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_232 -> p_231 [label=""];
p_231 [label="Simplify ((:var_0:.incoming = None\l \|\| (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).outgoing\l = None)\l \|\| (Destruct(Twist, 0,\l Option.get\l (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \>= 0\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_231 -> p_230 [label=""];
p_230 -> p_z3_1378 [label="",fontname="courier",fontsize=14];
p_z3_1378 [label="(let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2821 (get.Some.0_2823 (inco…",shape=box,fontname="courier",fontsize=14];p_230 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l((state.incoming = None\l \|\| (if state.incoming = None then state\l else\l if Is_a(Sensor, Option.get state.incoming)\l then if state.mode = Driving then … else …\l else if state.mode = Driving then … else …).outgoing\l = None)\l \|\| (Destruct(Twist, 0,\l Option.get\l (if state.incoming = None then state\l else\l if Is_a(Sensor, Option.get state.incoming)\l then if state.mode = Driving then … else …\l else if state.mode = Driving then … else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \>= 0)",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_284 [label="List.fold_right anon_fun.get_min_range.0\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max\l(Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_284 [label="calls"];
call_281 [label="foldi ((Destruct(Sensor, 0, …)).Sensor_msgs.

We've proven that the model never creates negative linear speed in response to any incoming message - alternatively we can set `state.outgoing = None` as a premise, proving that an empty `outgoing` message is never filled with a `Twist` with negative velocity:

In [21]:
theorem never_goes_back_alt state = 
  state.outgoing = None
  ==>
  no_moving_back (one_step state).outgoing  

val never_goes_back_alt : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.009s details Expand smt_stats num checks 2 arith assert lower 3 rlimit count 2521 mk clause 51 mk bool var 265 arith assert upper 10 datatype splits 27 decisions 53 propagations 178 conflicts 23 datatype accessor ax 58 minimized lits 5 arith assert diseq 2 datatype constructor ax 63 num allocs 933708233 added eqs 495 del clause 5 arith eq adapter 5 memory 32.130000 max memory 32.130000 Expand start[0.009s]
 :var_0:.outgoing = None
 ==> (if (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if ….mode = Driving then … else …
 else if ….mode = Driving then … else …).outgoing
 = None
 then true
 else
 (Destruct(Twist, 0, …)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x
 >= 0) simplify into (not (:var_0:.outgoing = None)
 || (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).outgoing
 = None)
|| (Destruct(Twist, 0,
 Option.get
 (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x
 >= 0 expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2979 (get.Some.0_2981 (inco… digraph "proof" {
p_238 [label="Start (:var_0:.outgoing = None\l ==\> (if (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if ….mode = Driving then … else …\l else if ….mode = Driving then … else …).outgoing\l = None\l then true\l else\l (Destruct(Twist, 0, …)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \>= 0)\l :time 0.009s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_238 -> p_237 [label=""];
p_237 [label="Simplify ((not (:var_0:.outgoing = None)\l \|\| (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).outgoing\l = None)\l \|\| (Destruct(Twist, 0,\l Option.get\l (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \>= 0\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_237 -> p_236 [label=""];
p_236 -> p_z3_1396 [label="",fontname="courier",fontsize=14];
p_z3_1396 [label="(let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_2979 (get.Some.0_2981 (inco…",shape=box,fontname="courier",fontsize=14];p_236 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l((not (state.outgoing = None)\l \|\| (if state.incoming = None then state\l else\l if Is_a(Sensor, Option.get state.incoming)\l then if state.mode = Driving then … else …\l else if state.mode = Driving then … else …).outgoing\l = None)\l \|\| (Destruct(Twist, 0,\l Option.get\l (if state.incoming = None then state\l else\l if Is_a(Sensor, Option.get state.incoming)\l then if state.mode = Driving then … else …\l else if state.mode = Driving then … else …).outgoing)).Geometry_msgs.twist_linear.Geometry_msgs.vector3_x\l \>= 0)",shape=box,style=filled,color="cyan",fontname="courier",fontsize=14];
call_229 [label="foldi ((Destruct(Sensor, 0, …)).Sensor_msgs.laserScan_range_max, 0) None\lanon_fun.get_min_and_direction.0\l(Destruct(Sensor, 0, …)).Sensor_msgs.laserScan_ranges",shape=box,style=filled,color="yellow",fontname="courier",fontsize=14];
goal -> call_229 [label="calls"];
call_180 [label="List.length\l(Destruct(Sensor, 0, Option.get …)).Sen

# 4. Inductive proofs. Stopping near objects.

As a final formal verification goal, we want to be able to prove that the robot stops and starts turning if one of the values in the scanner `ranges` is lower than 0.2 meters. In general, reasoning about variable-sized lists requires inductive proofs - and these might require proving some lemmas to guide Imandra to the proof. So, we will first try to prove a simpler version of the theorem - if all the ranges in the incoming laser scan message are less than 0.2 meters, then we definitely transition to the `Turning` state. We'll try to encode our theorem using `List.for_all` standard function:

In [22]:
verify ( fun state ->
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  List.for_all (fun x -> x < 20000) data.laserScan_ranges    
  ) ==> (one_step state).mode = Turning
)

- : state -> bool = <fun>
module CX : sig val state : state end


We have failed to prove the statement and Imandra has created a counterexample `CX` module for us. Examining the counterexample state we notice that the incoming `laserScan_ranges list` is empty.

In [23]:
CX.state

- : state =
{mode = Turning; min_range = Some 7719; direction = None;
 incoming =
  Some
   (Sensor
     {Sensor_msgs.laserScan_range_min = 3; laserScan_range_max = 25001;
      laserScan_ranges = []});
 outgoing = None}


Adding the extra requirement that the list is not `[]`, we successfully verify the statement:

In [24]:
theorem stopping_if_for_all state =
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  data.laserScan_ranges <> []
  && List.for_all (fun x -> x < 20000) data.laserScan_ranges    
  ) ==> (one_step state).mode = Turning

val stopping_if_for_all : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 4 definitions 0 inductions 0 search_time 0.017s details Expand smt_stats arith offset eqs 5 num checks 10 arith assert lower 36 arith pivots 18 rlimit count 7652 mk clause 67 datatype occurs check 1584 mk bool var 571 arith assert upper 46 datatype splits 49 decisions 361 arith add rows 25 propagations 284 conflicts 27 arith fixed eqs 9 datatype accessor ax 56 minimized lits 2 arith conflicts 3 arith assert diseq 9 datatype constructor ax 301 num allocs 1031543182 final checks 12 added eqs 1385 del clause 35 arith eq adapter 18 memory 39.270000 max memory 39.270000 Expand start[0.017s]
 if :var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming)
 && Is_a(Clock, Option.get :var_0:.incoming)
 then true
 else
 not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && List.for_all anon_fun.stopping_if_for_all.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 ==> (if :var_0:.incoming = None then :var_0:
 else if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).mode
 = Turning simplify into ((:var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming) && Is_a(Clock, Option.get :var_0:.incoming))
 || not
 (not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && List.for_all anon_fun.stopping_if_for_all.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges))
|| (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).mode
 = Turning expansions [] rewrite_steps forward_chaining unroll expr (let ((a!1 (Sensor_msgs.laserScan_range_max_2143
 (get.Sensor.0_3143 (get.Some.0_3145 (i… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_range_max_2143
 (get.Sensor.0_3143 (get.Some.0_3145 (i… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_3143 (get.Some.0_3145 (inco… expansions unroll expr (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_3143 (get.Some.0_3145 (inco… expansions unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_3143 (get.Some.0_3145 (inco… digraph "proof" {
p_252 [label="Start (if :var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming)\l then true\l else\l not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && List.for_all anon_fun.stopping_if_for_all.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l ==\> (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).mode\l = Turning\l :time 0.017s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_252 -> p_251 [label=""];
p_251 [label="Simplify (((:var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming))\l \|\| not\l (not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && List.for_all anon_fun.stopping_if_for_all.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges))\l \|\| (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).mode\l = Turning\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_251 -> p_250 [label=""];
p_250 [label="Unroll ([List.fold_right anon_fun.get_min_range.0\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_range_max\l (Destruct(Sensor, 0, Option.get …)).Sensor_msgs.laserScan_ranges],\l [[]])",shape=box,style=filled,fontname="courier",fontsize=14];
p_250 -> p_249 [label=""];
p_249 [label="Unroll ([foldi\l ((Destruct(Sensor, 0, …)).Sensor_msgs.laserScan_range_max

Imandra successfully proves the `stopping_if_for_all` theorem, but our ultimate goal is to prove the theorem when **some** of the values in `laserScan_ranges` are less than the cutoff. If we simply try to replace the `List.for_all` with `List.exists`, Imandra will fail to either prove or disprove the theorem.  The inductive structure of this proof is too complex for Imandra to figure out automatically without any hints from the user. We need to help it with the overall logic of the proof. To do that we will break this final theorem into several smaller steps, making a rough "sketch" of the inductive proof we want and and ask Imandra to fill in the gaps.

As a first step, we extract the anonymous threshold function and prove a lemma that if the `get_min_range` function returns a value satisfying the threshold, then the conclusion about the `one_step` function holds:

In [25]:
let under_threshold x = x < 20000

lemma lemma1 state =
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  data.laserScan_ranges <> []
  && under_threshold (get_min_range data.laserScan_range_max data.laserScan_ranges)
  ) ==> (one_step state).mode = Turning

val under_threshold : int -> bool = <fun>
val lemma1 : state -> bool = <fun>


Proved proof call graph proof summary full graph ground_instances 0 definitions 0 inductions 0 search_time 0.009s details Expand smt_stats num checks 2 arith assert lower 6 arith pivots 3 rlimit count 2362 mk clause 45 mk bool var 244 arith assert upper 17 datatype splits 26 decisions 60 arith add rows 1 propagations 101 conflicts 16 arith fixed eqs 2 datatype accessor ax 49 minimized lits 1 arith conflicts 1 arith assert diseq 1 datatype constructor ax 57 num allocs 1084619250 added eqs 361 del clause 25 arith eq adapter 5 memory 42.980000 max memory 42.980000 Expand start[0.009s]
 if :var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming)
 && Is_a(Clock, Option.get :var_0:.incoming)
 then true
 else
 not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && List.fold_right anon_fun.get_min_range.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_range_max
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 < 20000
 ==> (if :var_0:.incoming = None then :var_0:
 else if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).mode
 = Turning simplify into ((:var_0:.incoming = None
 || Is_a(Some, :var_0:.incoming) && Is_a(Clock, Option.get :var_0:.incoming))
 || not
 (not
 ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges
 = [])
 && not
 (20000 <=
 List.fold_right anon_fun.get_min_range.0
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_range_max
 (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges)))
|| (if :var_0:.incoming = None then :var_0:
 else
 if Is_a(Sensor, Option.get :var_0:.incoming)
 then if :var_0:.mode = Driving then … else …
 else if :var_0:.mode = Driving then … else …).mode
 = Turning expansions [] rewrite_steps forward_chaining unsat (let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_3242 (get.Some.0_3244 (inco… digraph "proof" {
p_255 [label="Start (if :var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming)\l then true\l else\l not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && List.fold_right anon_fun.get_min_range.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_range_max\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l \< 20000\l ==\> (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming) then … else …).mode\l = Turning\l :time 0.009s)",shape=box,style=filled,fontname="courier",fontsize=14];
p_255 -> p_254 [label=""];
p_254 [label="Simplify (((:var_0:.incoming = None\l \|\| Is_a(Some, :var_0:.incoming)\l && Is_a(Clock, Option.get :var_0:.incoming))\l \|\| not\l (not\l ((Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges\l = [])\l && not\l (20000 \<=\l List.fold_right anon_fun.get_min_range.0\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_range_max\l (Destruct(Sensor, 0, Option.get :var_0:.incoming)).Sensor_msgs.laserScan_ranges)))\l \|\| (if :var_0:.incoming = None then :var_0:\l else\l if Is_a(Sensor, Option.get :var_0:.incoming)\l then if :var_0:.mode = Driving then … else …\l else if :var_0:.mode = Driving then … else …).mode\l = Turning\l :expansions [] :rw [] :fc [])",shape=box,style=filled,fontname="courier",fontsize=14];
p_254 -> p_253 [label=""];
p_253 -> p_z3_1039 [label="",fontname="courier",fontsize=14];
p_z3_1039 [label="(let ((a!1 (Sensor_msgs.laserScan_ranges_2144
 (get.Sensor.0_3242 (get.Some.0_3244 (inco…",shape=box,fontname="courier",fontsize=14];p_253 [label="unsat",shape=box,style=filled,fontname="courier",fontsize=14];
}
 Load graph Loading.. digraph "call graph" {
goal [label="not\l(((state.incoming = None\l \|\| Is_a(Some, state.incoming) && Is_a(Clock, Option.get state.incoming))\l \|\| not\l (not\l ((Destruct(Sensor, 0, Option.get state.incomi

Next, we prove a "bridge" lemma that translates between the `get_min_range`
concept and the `List.exists` concept for the `under_threshold` function. Then, we `[@@apply]` the two lemmas above to prove our final theorem with the `List.exists` condition: these proofs require induction - to tell Imandra to use induction one should add the `[@@induct]` attribute to the theorem declaration:

In [28]:
lemma bridge max lst =
  List.exists under_threshold lst ==> under_threshold (get_min_range max lst)
  [@@induct]

theorem stopping_if_exists state =
  let open Sensor_msgs in
  match state.incoming with None | Some (Clock _ ) -> true 
  | Some ( Sensor data ) ->
  (  data.laserScan_ranges <> []
  && List.exists under_threshold data.laserScan_ranges
  ) ==> (one_step state).mode = Turning
[@@apply lemma1 state]
[@@apply bridge
    (match state.incoming with Some (Sensor data) -> data.laserScan_range_max | _ -> 0)
    (match state.incoming with Some (Sensor data) -> data.laserScan_ranges | _ -> []) ]
[@@induct]

val bridge : int -> int list -> bool = <fun>
Goal:

List.exists under_threshold lst ==> under_threshold (get_min_range max lst).

1 nontautological subgoal.

Subgoal 1:

 H0. List.exists under_threshold lst
 H1. 20000 <= List.fold_right anon_fun.get_min_range.0 max lst
|---------------------------------------------------------------------------
 false

Verified up to bound 5 (after 0.011s).

Must try induction.

Induction scheme:

 φ [] max && (lst <> [] && φ (List.tl lst) max ==> φ lst max).

2 nontautological subgoals.

Subgoal 1.2:

 H0. List.exists under_threshold []
 H1. 20000 <= List.fold_right anon_fun.get_min_range.0 max []
|---------------------------------------------------------------------------
 false

But simplification reduces this to true, using the definition of List.exists.

Subgoal 1.1:

 H0. lst <> []
 H1. not (List.exists under_threshold (List.tl lst))
     || not
        (20000 <= List.fold_right anon_fun.get_min_range.0 max (List.tl lst))
 H2. List.exists under_t

ground_instances,0
definitions,5
inductions,1
search_time,0.073s
into,true
expansions,"[List.fold_right, List.exists, List.fold_right, List.exists]"
rewrite_steps,
forward_chaining,
into,true
expansions,List.exists
rewrite_steps,
